In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
train.head()

In [ ]:
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")
test.head()

In [ ]:
# 学習用データ

# axis=1方向(列方向)に見て、"id"と"target"の列を除外
X = train.drop(["id", "target"], axis=1)

X.head()

In [ ]:
# 学習用データ

y = train["target"]
y.head()

In [ ]:
# テストデータ

X_test = test.drop("id", axis=1)
X_test.head()

In [ ]:
# pd.dataframe型だとxgboost lightgmb にデータを突っ込んだ時、まれにエラーが出る
# np.array型にしておけば絶対エラーは出ないので、np.array型に変換しておく

X = np.array(X)
y = np.array(y)

X_test = np.array(X_test)

学習用データ  
X, y  
  
テストデータ  
X_test

# Modeling

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb

In [ ]:
# 学習用データのX, yをさらに学習用と精度検証用に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=912)

print(X_train.shape)
print(X_val.shape)

# XGBoost (Regressor, 回帰) 

In [ ]:
# まずパラメータを設定する
# First, we set the parameters

params_xgb = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "max_depth": 8,
    "eta": 0.07,
    "random_state": 234
}

objective (目的関数)  
・reg:squarederror <- for regression, 回帰用, 最小二乗誤差  
・reg:logistic <- for logistic regression, for binary classification, ロジスティック回帰用, 二値分類用    
・multi:softmax <- for multi class classification, 多値分類用  
(num_class) <- you have to set "num_class" if objective = multi:softmax. もしobjectiveにmulti:softmaxを選んだ時は、"num_class"も設定しなければならない  
  
eval_metric (評価指標)  
・rmse <- for reg, 回帰用, 平方平均二乗誤差  
・mae <- for reg, 回帰用, 平均絶対誤差  
・logloss <- for binary classification, 二値分類用  
・mlogloss <- for multi classification, 多値分類用  
  
max_depth (木の深さ)  
usually, 4 ~ 10. 普通4~10くらい。  
  
eta (学習率)  
usually, 0.05 ~ 0.12. 普通0.05 ~ 0.12くらい。

In [ ]:
# 次に、データをxgboost専用のデータ型に変換する
# Next, we convert the data to the xgboost-specific data type

d_train = xgb.DMatrix(X_train, label=y_train)
d_val = xgb.DMatrix(X_val, label=y_val)

d_test = xgb.DMatrix(X_test)

In [ ]:
# そして、学習させる。
# and start learning

model = xgb.train(params=params_xgb,    # set the parameters, パラメータをセット
                  dtrain=d_train,    # set the train data, 学習用データをセット
                  num_boost_round=10000,    # number of DecisionTrees, 決定木の本数
                  early_stopping_rounds=20,    # early_stopping, アーリーストッピングの回数
                  verbose_eval=10,    # Display the learning process every 10 times. 学習過程を10回ごとに表示
                  evals=[(d_train, "train"), (d_val, "val")])    # Set data for evaluation, 評価用データをセット

In [ ]:
# 最後にテストデータに対して予測を行う
# Finally, make predictions on the test data

# ntree_limit=model.best_ntree_limit <- Use the decision tree with the best accuracy for prediction.
# ntree_limit=model.best_ntree_limit < -最も精度の良い決定木を使う。という意味
prediction = model.predict(d_test, ntree_limit=model.best_ntree_limit)

In [ ]:
prediction

# Submission (提出)

In [ ]:
sample_sub = pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")
my_submission = sample_sub.copy()
my_submission

In [ ]:
my_submission["target"] = prediction
my_submission

In [ ]:
my_submission.to_csv("submission_xgb.csv", index=False)